<a href="https://colab.research.google.com/github/Yun-choice/DA_Archive/blob/main/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoBERT finetuning

In [1]:
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments

In [2]:
# 모델 및 토크나이저 로드
model_name = "skt/kobert-base-v1"  # 또는 다른 KoBERT 모델
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # num_labels는 분류할 클래스 수에 따라 변경

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# NSMC 영화 리뷰 데이터셋 준비
dataset = load_dataset("e9t/nsmc")

def tokenize_function(example):
    return tokenizer(example["document"])

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

README.md: 0.00B [00:00, ?B/s]

nsmc.py: 0.00B [00:00, ?B/s]

RuntimeError: Dataset scripts are no longer supported, but found nsmc.py

In [ ]:
# 데이터 길이를 똑같이 맞추기
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir="./results",          # 결과 저장 디렉토리
    learning_rate=5e-5,
    per_device_train_batch_size=64,  # 훈련 배치 크기
    per_device_eval_batch_size=64,   # 평가 배치 크기
    num_train_epochs=5,              # 훈련 에폭 수
    weight_decay=0.01,               # 가중치 감소
    eval_strategy="epoch",     # 에폭마다 평가 수행
    save_strategy="epoch",          # 에폭마다 모델 저장
    load_best_model_at_end=True,     # 훈련 종료 시 가장 좋은 모델 로드
    metric_for_best_model="accuracy", # 가장 좋은 모델을 선택하기 위한 메트릭
    push_to_hub=False,               # Hugging Face Hub에 모델 업로드 여부
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": np.mean(predictions == labels)}

# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 훈련 시작
trainer.train()

In [ ]:
# 평가 진행
trainer.evaluate()